In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = ["C7000","C1700","C4000","C5000","C1270","C2700","C2800","C7100","C1300","C1280","C1230","C1400","C7200","C2300","C1240","C8000","C7120","C1500","C1800","C6000","C1250","C8200","C1238","C1278","C1235","C1237","C7210","C2400","C1720","C4100","C1257","C1600","C1260","C2710","C0","C3200","C1234","C1246","C1267","C1256","C2190","C4200","C2600","C5200","C1370","C1248","C6100","C1820","C1900","C1236","C3700","C2570","C1580","C1245","C2500","C1570","C1283","C2380","C1732","C1728","C2170","C4120","C8210","C2561","C4500","C2150"]
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [5]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=[
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'STATUS',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS'])

In [6]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop(columns=["IS_SUCCESSFUL"])
y = application_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=200, activation='relu', input_dim=44))

# First hidden layer
model.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=50, activation='relu'))

#Third hidden layer
model.add(tf.keras.layers.Dense(units=25, activation='relu'))

#Fourth hidden layer
model.add(tf.keras.layers.Dense(units=10, activation='relu'))


# Output layer
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               9000      
                                                                 
 dense_1 (Dense)             (None, 100)               20100     
                                                                 
 dense_2 (Dense)             (None, 50)                5050      
                                                                 
 dense_3 (Dense)             (None, 25)                1275      
                                                                 
 dense_4 (Dense)             (None, 10)                260       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 35696 (139.44 KB)
Trainable params: 35696 

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'], )

In [10]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=50)

Epoch 1/50
515/515 [==============================] - 3s 2ms/step - loss: 0.1925 - accuracy: 0.7193
Epoch 2/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1864 - accuracy: 0.7294
Epoch 3/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1853 - accuracy: 0.7307
Epoch 4/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1847 - accuracy: 0.7317
Epoch 5/50
515/515 [==============================] - 2s 3ms/step - loss: 0.1841 - accuracy: 0.7319
Epoch 6/50
515/515 [==============================] - 2s 4ms/step - loss: 0.1837 - accuracy: 0.7338
Epoch 7/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1835 - accuracy: 0.7329
Epoch 8/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1832 - accuracy: 0.7337
Epoch 9/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1831 - accuracy: 0.7338
Epoch 10/50
515/515 [==============================] - 1s 2ms/step - loss: 0.1828 - accuracy: 0.7352

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.1871 - accuracy: 0.7256 - 504ms/epoch - 2ms/step
Loss: 0.1871228814125061, Accuracy: 0.7255976796150208


In [12]:
# Export our model to HDF5 file
# save model and architecture to single file
model.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
